In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
import os

# Path to the labels directory in Google Drive
labels_dir = '/content/drive/MyDrive/DATA_PLATES/Label'

# List all files in the directory
json_files = [f for f in os.listdir(labels_dir) if f.endswith('.json')]
print(json_files)


['5_5_2020_12h52m54s.json', '9_5_2020_21h31m54s.json', '16_5_2020_23h5m51s.json', '10_5_2020_8h11m58s.json', '17_5_2020_17h53m1s.json', '9_5_2020_17h6m52s.json', '21_5_2020_17h34m58s.json', '14_5_2020_20h5m43s.json', '21_5_2020_20h25m9s.json', '14_5_2020_19h57m27s.json', '16_5_2020_22h18m9s.json', '21_5_2020_10h22m41s.json', '14_5_2020_16h52m24s.json', '17_5_2020_16h39m15s.json', '10_5_2020_16h53m26s.json', '3_5_2020_21h15m57s.json', '8_5_2020_18h9m28s.json', '23_5_2020_14h19m55s.json', '14_5_2020_22h14m32s.json', '10_5_2020_15h22m33s.json', '4_5_2020_21h43m18s.json', '17_5_2020_9h21m16s.json', '9_5_2020_22h13m30s.json', '17_5_2020_16h33m11s.json', '22_5_2020_21h20m54s.json', '21_5_2020_17h12m50s.json', '9_5_2020_20h39m2s.json', '10_5_2020_11h16m12s.json', '21_5_2020_10h16m7s.json', '4_5_2020_9h35m25s.json', '21_5_2020_9h21m24s.json', '9_5_2020_19h51m5s.json', '22_5_2020_21h28m18s.json', '23_5_2020_12h40m27s.json', '14_5_2020_23h57m12s.json', '22_5_2020_21h3m5s.json', '8_5_2020_20h8m49

In [16]:
import json
import os

def json_to_yolo_format(json_path, output_dir):
    with open(json_path) as f:
        data = json.load(f)

    # Iterate through images
    for img_id, img_info in data["_via_img_metadata"].items():
        img_filename = img_info["filename"]
        img_size = img_info["size"]

        # Create corresponding txt file
        txt_filename = os.path.join(output_dir, img_filename.replace('.jpg', '.txt'))

        with open(txt_filename, 'w') as txt_file:
            # Process each region
            for region in img_info["regions"]:
                shape_attributes = region["shape_attributes"]
                region_attributes = region["region_attributes"]

                # Convert polygon to bounding box
                x_min = min(shape_attributes["all_points_x"])
                x_max = max(shape_attributes["all_points_x"])
                y_min = min(shape_attributes["all_points_y"])
                y_max = max(shape_attributes["all_points_y"])

                # Compute YOLO format coordinates
                x_center = (x_min + x_max) / 2 / img_size
                y_center = (y_min + y_max) / 2 / img_size
                width = (x_max - x_min) / img_size
                height = (y_max - y_min) / img_size

                # Use label as class_id
                class_id = region_attributes["label"]

                # Write to txt file
                txt_file.write(f"{class_id} {x_center} {y_center} {width} {height}\n")

def process_folder(input_folder, output_folder):
    # Create output directory if it does not exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Iterate through all JSON files in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith('.json'):
            json_path = os.path.join(input_folder, filename)
            json_to_yolo_format(json_path, output_folder)

# Example usage
input_folder = '/content/drive/MyDrive/DATA_PLATES/Label'
output_folder = '/content/json_file1'
process_folder(input_folder, output_folder)
